# Block Encoding

High level bloqs for defining bloq encodings and operations on block encodings.

A block encoding of the Hamiltonian $H$ obeys:

$$
    H = (\langle G|_a\otimes I_s U |G\rangle_a \otimes I_s),
$$

where $a$ is an ancilla register and $s$ is the system register, $U$ is a unitary sometimes
called a signal oracle and encodes $H$ in its top right corner, while
$|G\rangle_a$ is called the signal state. Oftentimes $U$ is a SELECT oracle implementing
$$
    U|G\rangle_a|\psi\rangle_s = |G\rangle_a H |\psi\rangle_s,
$$
and $|G\rangle = \mathrm{PREPARE}|0\rangle_a$, which for a LCU representation
of $H$ would prepare a superposition over the Hamiltonian LCU coefficients.
However, there are other cases where it is not convenient to factorize the
block encoding so cleanly (single-factorizated and double factorizated)
chemistry Hamiltonians, or more generally we may only have access to $U$.
In that case $|G\rangle = |0\rangle_a$.

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `LCUBlockEncoding`
LCU based block encoding from using SELECT and PREPARE oracles.

Builds the block encoding via
$$
    \mathcal{B}[H] = \mathrm{PREPARE}^\dagger \cdot \mathrm{SELECT} \cdot \mathrm{PREPARE},
$$
where
$$
    \mathrm{PREPARE} |0\rangle_a = \sum_l \sqrt{\frac{w_l}{\lambda}} |l\rangle_a,
$$
and
$$
    \mathrm{SELECT} |l\rangle_a|\psi\rangle_s = |l\rangle_a U_l |\psi\rangle_s.
$$

The ancilla register is at least of size $\log L$. In our implementations we
typically split the ancilla registers into selection registers (i.e.  the
$l$ registers above) and junk registers which are extra qubits needed by
state preparation but not controlled upon during SELECT.

#### Parameters
 - `select`: The bloq implementing the `SelectOracle` interface.
 - `prepare`: The bloq implementing the `PrepareOracle` interface. 

#### Registers
 - `selection`: The combined selection register.
 - `junk`: Additional junk registers not prepared upon.
 - `system`: The combined system register. 

#### References
 - [Hamiltonian Simulation by Qubitization](https://quantum-journal.org/papers/q-2019-07-12-163/).     Sec 3.1, page 7 and 8 for high level overview and definitions. A     block encoding is called a standard form encoding there.


In [ ]:
from qualtran.bloqs.block_encoding import LCUBlockEncoding

### Example Instances

In [ ]:
from qualtran.bloqs.hubbard_model import PrepareHubbard, SelectHubbard

dim = 3
select = SelectHubbard(x_dim=dim, y_dim=dim)
prepare = PrepareHubbard(x_dim=dim, y_dim=dim, t=1, mu=4)
black_box_block_bloq = LCUBlockEncoding(select=select, prepare=prepare)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([black_box_block_bloq],
           ['`black_box_block_bloq`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
black_box_block_bloq_g, black_box_block_bloq_sigma = black_box_block_bloq.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(black_box_block_bloq_g)
show_counts_sigma(black_box_block_bloq_sigma)

## `ChebyshevPolynomial`
Block encoding of $T_j[H]$ where $T_j$ is the $j$-th Chebyshev polynomial.

Here H is a Hamiltonian with spectral norm $|H| \le 1$, we assume we have
an $n_L$ qubit ancilla register, and assume that $j > 0$ to avoid block
encoding the identity operator.

Recall:

\begin{align*}
    T_0[H] &= \mathbb{1} \\
    T_1[H] &= H \\
    T_2[H] &= 2 H^2 - \mathbb{1} \\
    T_3[H] &= 4 H^3 - 3 H \\
    &\dots
\end{align*}

#### Parameters
 - `block_encoding`: Block encoding of a Hamiltonian $H$, $\mathcal{B}[H]$.
 - `order`: order of Chebychev polynomial. 

#### References
 - [Quantum computing enhanced computational catalysis](https://arxiv.org/abs/2007.14460).     Page 45; Theorem 1.


In [ ]:
from qualtran.bloqs.block_encoding import ChebyshevPolynomial

### Example Instances

In [ ]:
from qualtran.bloqs.block_encoding import LCUBlockEncoding
from qualtran.bloqs.hubbard_model import PrepareHubbard, SelectHubbard

dim = 3
select = SelectHubbard(x_dim=dim, y_dim=dim)
prepare = PrepareHubbard(x_dim=dim, y_dim=dim, t=1, mu=4)
black_box_block_bloq = LCUBlockEncoding(select=select, prepare=prepare)
chebyshev_poly = ChebyshevPolynomial(black_box_block_bloq, order=3)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([chebyshev_poly],
           ['`chebyshev_poly`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
chebyshev_poly_g, chebyshev_poly_sigma = chebyshev_poly.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(chebyshev_poly_g)
show_counts_sigma(chebyshev_poly_sigma)